<a href="https://colab.research.google.com/github/Glitch0110/AI-GiganciProgramowania/blob/main/L21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import random
import math
#1. Wczytywanie danych treningowych

In [ ]:
#2. Trenowanie klasyfikatora bayesa

In [ ]:
#3. Klasyfikacja wiadomości

In [ ]:
#4. Główna funkcja